## Conversion from convolution filter to binary filter

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


2021-12-20 02:38:09.788243: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 02:38:09.788286: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
assert tf.__version__ >= "2.7.0"

In [3]:
tf.config.run_functions_eagerly(True)
tf.executing_eagerly()
tf.data.experimental.enable_debug_mode()


In [4]:
# Model / data parameters
num_classes = 20
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='coarse')

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to one-hot representations
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
model_full_precision = keras.Sequential(
    [
        keras.Input(shape=input_shape),

        

        layers.Conv2D(6, kernel_size=(5, 5), activation="relu", padding = "valid"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        
        layers.Conv2D(16, kernel_size=(5, 5), activation="relu", padding = "valid"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        
        
        
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_full_precision.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 6)         456       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 6)        0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 14, 14, 6)        24        
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                      

In [11]:
batch_size = 64
epochs = 5

model_full_precision.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.TopKCategoricalAccuracy(k=3)],run_eagerly=True)

model_full_precision.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/5
  1/704 [..............................] - ETA: 1:15 - loss: 3.3888 - categorical_accuracy: 0.0625 - top_k_categorical_accuracy: 0.1406

2021-12-20 02:38:55.826063: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 552960000 exceeds 10% of free system memory.


704/704 [==============================] - 38s 54ms/step - loss: 2.3123 - categorical_accuracy: 0.2913 - top_k_categorical_accuracy: 0.5278 - val_loss: 2.2041 - val_categorical_accuracy: 0.3196 - val_top_k_categorical_accuracy: 0.5746
Epoch 2/5
704/704 [==============================] - 38s 54ms/step - loss: 1.9472 - categorical_accuracy: 0.3959 - top_k_categorical_accuracy: 0.6510 - val_loss: 2.0870 - val_categorical_accuracy: 0.3628 - val_top_k_categorical_accuracy: 0.6152
Epoch 3/5
704/704 [==============================] - 38s 54ms/step - loss: 1.7743 - categorical_accuracy: 0.4462 - top_k_categorical_accuracy: 0.7014 - val_loss: 1.9650 - val_categorical_accuracy: 0.3980 - val_top_k_categorical_accuracy: 0.6542
Epoch 4/5
704/704 [==============================] - 38s 55ms/step - loss: 1.6274 - categorical_accuracy: 0.4916 - top_k_categorical_accuracy: 0.7414 - val_loss: 1.9954 - val_categorical_accuracy: 0.4000 - val_top_k_categorical_accuracy: 0.6506
Epoch 5/5
704/704 [===========

In [12]:

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model_full_precision.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
79/79 [==============================] - 2s 27ms/step - loss: 1.9678 - categorical_accuracy: 0.4083 - top_k_categorical_accuracy: 0.6607
test loss, test acc: [1.9678025245666504, 0.4083000123500824, 0.6607000231742859]
